In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [15]:
# 导入所需的库
import os
import gradio as gr
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_openai import OpenAIEmbeddings
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import Qdrant
from langchain.memory import ConversationSummaryMemory
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import Docx2txtLoader
from langchain_community.document_loaders import TextLoader

In [55]:
# 设置OpenAI API密钥
os.environ["MOONSHOT_API_KEY"] = 'sk-tRUEvHARtePCB5t83Lq73qwFrPLIc9RPgRFLQM2CneZUjlNB'

### 加载Documents

In [4]:
folder = "OneFlower"
base_dir = folder # 文档的存放目录
documents = []
for file in os.listdir(base_dir):
    file_path = os.path.join(base_dir, file)
    if file.endswith('.pdf'):
        loader = PyPDFLoader(file_path)
        documents.extend(loader.load())
    elif file.endswith('.docx') or file.endswith('.doc'):
        loader = Docx2txtLoader(file_path)
        documents.extend(loader.load())
    elif file.endswith('.txt'):
        loader = TextLoader(file_path)
        documents.extend(loader.load())

In [5]:
len(documents)
documents[:10]

34

[Document(metadata={'source': 'OneFlower/易速鲜花运营指南.docx'}, page_content='易速鲜花网站是一个服务型的网站，提供的是一个平台和在这个平台上的服务。纵观整个互联网，最能盈利的网站就是那些提供服务的网站。在经过了互联网泡沫之后，互联网产生的新的一种盈利的方式就是提供各种各样的服务。\n\n网站提供的是服务，而不是现在互联网上大多数网站提供的是信息，用户可以到我们的网站来进行一些活动，做他们想做的事情，我们提供的是这个平台，就象市场一样，有很多卖菜的摊子，大家可以进去买菜。在社会高速发展的今天，相信大家都很清楚的了解，没有了互联网我们的生活将会是什么样子，工作、生活、学习等等都已经离不开互联网，但是即便是这样我觉的互联网还是没有充分的利用起来。现在的互联网的用处只不过是大家获得自己需要的信息的一种途径，互联网从产生那一天起就是为了信息的共享这个目的，并不是在上面进行自己的活动，例如组织起来打篮球等等。即便是现在已经有众多的社区、交友型的网站但是发展的也够理想，但是这些网站依旧很火爆。而我们的想法是要把互联网变成一个活动或者是日常生活的场所，在上面你也许获得不到信息或者一些资源，但是你能获得的是跟你生活息息相关的东西。所以网站定位的使用人群是哪些人，必须能够上网并且接受这种全新的生活方式跟社会活动方式。这个思想这样说选好消费人群即针对哪种客户提供什么样的服务。\n\n网站有很多类型的或者是不同功能，而我们做的只是一个分类，网站定位很重要。拼房，现在很多有关房屋的网站都会有人发帖进行合租，但是并没有提供完善的平台，用户对这方面的需求不是很集中，网站就充分的解决了这个问题，给大家一个很好的平台很容易的拼起来。网站就可以提供基于这些拼的平台，对他们进行详细的分类，并且提供了方便用户使用的功能，提供了安全的机制，并且提供了更加方便用户拼的各种功能。加之给大家提供了一个信任、时尚、活跃的环境。网站的各种拼的方式，有助于用户更加好的生活，尝试这种新的生活方式给他们带来的快乐、方便、节省。例如网站的拼项目，其实就是项目合作，这就给了他们寻找一起能合作的项目的平台，扩大人脉。所以网站总的来说就使提供了各种概念的服务，这个平台也同样为用户的安全性、易用性提供了很好的解决方式。根据用户的需求，用户同样也可以自己一些新型的需求。

### 文本的分割

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=0)
all_splits = text_splitter.split_documents(documents)

In [7]:
len(all_splits)
all_splits[:10]

112

[Document(metadata={'source': 'OneFlower/易速鲜花运营指南.docx'}, page_content='易速鲜花网站是一个服务型的网站，提供的是一个平台和在这个平台上的服务。纵观整个互联网，最能盈利的网站就是那些提供服务的网站。在经过了互联网泡沫之后，互联网产生的新的一种盈利的方式就是提供各种各样的服务。'),
 Document(metadata={'source': 'OneFlower/易速鲜花运营指南.docx'}, page_content='网站提供的是服务，而不是现在互联网上大多数网站提供的是信息，用户可以到我们的网站来进行一些活动，做他们想做的事情，我们提供的是这个平台，就象市场一样，有很多卖菜的摊子，大家可以进去买菜。在社会高速发展的今天，相信大家都很清楚的了解，没有了互联网我们的生活将会是什么样子，工作、生活、学习等等都已经离不开互联网，但是即便是这样我觉的互联网还是没有充分的利用起来。现在的互联网的用处只不过是大家获得自己'),
 Document(metadata={'source': 'OneFlower/易速鲜花运营指南.docx'}, page_content='需要的信息的一种途径，互联网从产生那一天起就是为了信息的共享这个目的，并不是在上面进行自己的活动，例如组织起来打篮球等等。即便是现在已经有众多的社区、交友型的网站但是发展的也够理想，但是这些网站依旧很火爆。而我们的想法是要把互联网变成一个活动或者是日常生活的场所，在上面你也许获得不到信息或者一些资源，但是你能获得的是跟你生活息息相关的东西。所以网站定位的使用人群是哪些人，必须能够上网并且接受这种全'),
 Document(metadata={'source': 'OneFlower/易速鲜花运营指南.docx'}, page_content='新的生活方式跟社会活动方式。这个思想这样说选好消费人群即针对哪种客户提供什么样的服务。'),
 Document(metadata={'source': 'OneFlower/易速鲜花运营指南.docx'}, page_content='网站有很多类型的或者是不同功能，而我们做的只是一个分类，网站定位很重要。拼房，现在很多有关房屋的网站都会有人发帖进行合租，但是并没有提供完善的平台，用户对这方面的

### 向量数据库

In [9]:
ollama_embedding=OllamaEmbeddings(model="nomic-embed-text")

vectorstore = Qdrant.from_documents(
    documents=all_splits, # 以分块的文档
    embedding=ollama_embedding,
    location=":memory:",  # in-memory 存储
    collection_name="my_documents" # 指定collection_name
)

In [13]:
text = "易速鲜花网站提供什么服务"
single_vector = ollama_embedding.embed_query(text)
single_vector[:10]
vectorstore.similarity_search(text, k=3)

[0.019558923,
 0.062161416,
 -0.17511658,
 -0.021494376,
 -0.020503124,
 0.025355589,
 -5.6143574e-05,
 -0.023982685,
 0.019788224,
 -0.019960182]

[Document(metadata={'source': 'OneFlower/易速鲜花员工手册.pdf', 'page': 3, '_id': '4f7b43ee5557466ca21745e7f8696902', '_collection_name': 'my_documents'}, page_content='乐土，享受易速鲜花带给您的美好。  \n易速鲜花， 欢迎您的加入。'),
 Document(metadata={'source': 'OneFlower/易速鲜花员工手册.pdf', 'page': 0, '_id': 'cc5b96df33fa4cedac1c10e79ae46a59', '_collection_name': 'my_documents'}, page_content='易速鲜花集团 \n \n \n \n \n \n \n \n \n \n \n易速鲜花 服务中心  \n易速鲜花 股份有限公司  易\n速\n鲜\n花\n员\n工\n手\n册'),
 Document(metadata={'source': 'OneFlower/易速鲜花员工手册.pdf', 'page': 3, '_id': '16c1e92419e34e10b5ace49816d5e191', '_collection_name': 'my_documents'}, page_content='易速鲜花，她以高品位的服务，超前的经营理念，尖端的营 销策\n略和丰富的艺术气质，旨在满足大众的需求，创造独一无二的网上购\n花体验。  \n易速鲜花，她是古老与现代的完美融合，她将带您体验夜郎古国\n的历史风貌，感受穿越时空的神奇。购花不再只是交易，更是一次文\n化之旅、美食之旅、音乐之旅，一场美妙体验的新境界。  \n易速鲜花，她超越了传统网络鲜花运营商单一的经营模式，她将')]

### 初始化LLM

In [25]:
llm = ChatOpenAI(
    api_key=os.environ["MOONSHOT_API_KEY"],
    base_url="https://api.moonshot.cn/v1",
    model="moonshot-v1-8k"
)

In [17]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x12678e390>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x12678fef0>, root_client=<openai.OpenAI object at 0x12678cb90>, root_async_client=<openai.AsyncOpenAI object at 0x12678e3c0>, openai_api_key=SecretStr('**********'), openai_api_base='https://api.moonshot.cn/v1', openai_proxy='')

### 初始化Memory

In [26]:
memory = ConversationSummaryMemory(
    llm=llm,
    memory_key="chat_history",
    return_messages=True
)

In [19]:
memory

ConversationSummaryMemory(llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x12678e390>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x12678fef0>, root_client=<openai.OpenAI object at 0x12678cb90>, root_async_client=<openai.AsyncOpenAI object at 0x12678e3c0>, openai_api_key=SecretStr('**********'), openai_api_base='https://api.moonshot.cn/v1', openai_proxy=''), return_messages=True, memory_key='chat_history')

### 设置Retrieval Chain

In [33]:
# 初始化对话历史
conversation_history = ""

retriever = vectorstore.as_retriever()
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory,
    verbose=True,
)

In [28]:
qa

ConversationalRetrievalChain(memory=ConversationSummaryMemory(llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x1283f85f0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x1283f9eb0>, root_client=<openai.OpenAI object at 0x1275fec00>, root_async_client=<openai.AsyncOpenAI object at 0x1267aac00>, model_name='moonshot-v1-8k', openai_api_key=SecretStr('**********'), openai_api_base='https://api.moonshot.cn/v1', openai_proxy=''), return_messages=True, memory_key='chat_history'), combine_docs_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=ChatPromptTemplate(input_variables=['context', 'question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="Use the following pieces of context to answer the user's question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}")), HumanMessagePromptTemplate(prompt=PromptTe

In [34]:
response = qa("玫瑰花的花语是什么？")
response




> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

system: The human inquires about the meaning behind rose flowers. The AI explains that roses generally symbolize love, romance, and beauty, with different colors holding specific meanings, such as red for passion and deep love, pink for gentleness and appreciation, and white for purity and respect.
Follow Up Input: 玫瑰花的花语是什么？
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
易速鲜花，她以高品位的服务，超前的经营理念，尖端的营 销策
略和丰富的艺术气质，旨在满足大众的需求，创造独一无二的网上购
花体验。  
易速鲜花，她是古老与现代的完美融合，她将带您体验夜郎古国
的历史风貌，感受穿越时空的神奇。购花不再只是

{'question': '玫瑰花的花语是什么？',
 'chat_history': [SystemMessage(content='The human inquires about the meaning behind rose flowers. The AI explains that roses generally symbolize love, romance, and beauty, with different colors holding specific meanings, such as red for passion and deep love, pink for gentleness and appreciation, and white for purity and respect.')],
 'answer': '在花语中，玫瑰花朵通常代表爱情、浪漫和美丽的感情。不同颜色的玫瑰可能有不同的具体含义，例如红玫瑰通常象征着热烈的爱情，粉玫瑰可能代表温柔的爱情，黄玫瑰可能表示友谊或嫉妒等。但具体的含义可能会因文化和个人理解而有所不同。'}

In [37]:
response = qa("易速鲜花晨会宣言是什么？")
response



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

system: The human inquires about the meaning behind rose flowers, to which the AI explains that roses generally symbolize love, romance, and beauty, with different colors holding specific meanings, such as red for passion and deep love, pink for gentleness and appreciation, and white for purity and respect. The human then asks about the symbolism of roses in Chinese, and the AI reiterates that in the language of flowers, roses represent emotions of love, romance, and beauty, with red symbolizing passionate love, pink denoting tenderness, and yellow indicating friendship or jealousy, noting that meanings can vary by culture and personal interpretation.
Follow Up Input: 易速鲜花晨会宣言是什么？
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...




RateLimitError: Error code: 429 - {'error': {'message': 'Your account cnopfsecp7f6u2rg79ng<ak-eu1biefahiui11d57k51> request reached max request: 3, please try again after 1 seconds', 'type': 'rate_limit_reached_error'}}

In [30]:
def get_response(user_input):  # 这是为 Gradio 创建的新函数
        response = qa(user_input)
        # 更新对话历史
        conversation_history += f"你: {user_input}\nChatbot: {response['answer']}\n"
        return conversation_history

### 定义 Gradio 界面

In [31]:
interface = gr.Interface(
    fn=get_response,  # 使用我们刚刚创建的函数
    inputs="text",  # 输入是文本
    outputs="text",  # 输出也是文本
    live=False,  # 实时更新，这样用户可以连续与模型交互
    title="易速鲜花智能客服",  # 界面标题
    description="请输入问题，然后点击提交。"  # 描述
)
interface.launch(share=True)  # 启动 Gradio 界面

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://6b2c3a01ad1851ca12.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/Users/cal/cal/code/ai/torch/langchain-in-action/.venv/lib/python3.12/site-packages/gradio/queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/cal/cal/code/ai/torch/langchain-in-action/.venv/lib/python3.12/site-packages/gradio/route_utils.py", line 288, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/cal/cal/code/ai/torch/langchain-in-action/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 1931, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/cal/cal/code/ai/torch/langchain-in-action/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 1516, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [44]:
import sqlite3

import requests
from langchain_community.utilities.sql_database import SQLDatabase
from sqlalchemy import create_engine
from sqlalchemy.pool import StaticPool


def get_engine_for_chinook_db():
    """Pull sql file, populate in-memory database, and create engine."""
    url = "https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql"
    response = requests.get(url)
    sql_script = response.text

    connection = sqlite3.connect(":memory:", check_same_thread=False)
    connection.executescript(sql_script)
    return create_engine(
        "sqlite://",
        creator=lambda: connection,
        poolclass=StaticPool,
        connect_args={"check_same_thread": False},
    )


engine = get_engine_for_chinook_db()

db = SQLDatabase(engine)

In [45]:
db

In [54]:
# https://python.langchain.com/v0.2/docs/tutorials/sql_qa/#agents
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
tools = toolkit.get_tools()
tools

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x127548aa0>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x127548aa0>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x127548aa0>),
 QuerySQLCheckerTool(description='Use this tool to double check if your 

In [52]:
from langchain_core.messages import SystemMessage
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

SQL_PREFIX = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables."""

system_message = SystemMessage(content=SQL_PREFIX)


agent_executor = create_react_agent(llm, tools, state_modifier=system_message)

In [53]:
for s in agent_executor.stream(
    {"messages": [HumanMessage(content="Which country's customers spent the most?")]}
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'sql_db_list_tables:0', 'function': {'arguments': '{}', 'name': 'sql_db_list_tables'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 568, 'total_tokens': 581}, 'model_name': 'moonshot-v1-8k', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-bac4b716-4e67-4236-af68-3cdb76dc0c96-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {}, 'id': 'sql_db_list_tables:0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 568, 'output_tokens': 13, 'total_tokens': 581})]}}
----
{'tools': {'messages': [ToolMessage(content='Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track', name='sql_db_list_tables', tool_call_id='sql_db_list_tables:0')]}}
----
{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'sq

RateLimitError: Error code: 429 - {'error': {'message': 'Your account cnopfsecp7f6u2rg79ng<ak-eu1biefahiui11d57k51> request reached max request: 3, please try again after 1 seconds', 'type': 'rate_limit_reached_error'}}